## Imports

In [ ]:
import numpy as np
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import urllib
import requests
import random
import warnings
warnings.filterwarnings("ignore")

## Setting options for Selenium webdriver

In [ ]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 92.0 Win10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 91.0 Win10
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Firefox 90.0 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]    

## Scraping data for all hotels in Californa from Tripadvisor

In [ ]:
url = 'https://www.tripadvisor.com/Hotels-g28926-California-Hotels.html'
driver = webdriver.Chrome('chromedriver.exe', options = chrome_options)
driver.get(url)

In [ ]:
count=0
i=0
listing_url = []
title = []
rank = []
rating = []
review = []
amenities = []
rfeatures = []
rtypes = []
hclass = []
hstyle = []
description = []
lang = []
website = []
offers = []
packages = []
covid = []
locationrating = []
cleanrating = []
valuerating = []
servicerating = []
walkscore = []
restoscore = []
attractscore = []
image_count = []
roomtips = []
price = []

while True:
    
    time.sleep(5)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    for listing in soup.find_all('div', attrs={"class":"ui_column is-8 main_col allowEllipsis"}):
    
        if listing.find('span', attrs={"class":"ui_merchandising_pill sponsored_v2"})!=None:
            continue
        titletag = listing.find('a', attrs={"target":"_blank"})
        listing_url.append('https://www.tripadvisor.com'+titletag['href'])
        title.append(titletag.getText().strip())
        
        try:
            ranktag = listing.find('div', attrs={"class":"prw_rup prw_common_location_pop_index linespace is-shown-at-mobile"})
            rank.append(ranktag.find('div', attrs={"class":"popindex"}).getText().split()[0])
        except:
            rank.append(None)
            
        try:
            pricetag = listing.find('div', attrs={"class":"price-wrap"})
            price.append(pricetag.getText().split('$')[-1])
        except:
            price.append(None)
        
        
        headers = random.choice(headers_list)
        r = requests.Session()
        r.headers = headers
        listing_html = r.get(listing_url[i]).text
        lsoup = bs(listing_html, 'html.parser')
        
        reviewtag = lsoup.find('div', attrs={"class":"bSlOX Xe f P"})
        try:
            rating.append(reviewtag.find('span', attrs={"class":"bvcwU P"}).getText())
        except:
            rating.append(None)
        try:
            review.append(reviewtag.find('span', attrs={"class":"btQSs q Wi z Wc"}).getText().split()[0])
        except:
            review.append(None)
        
        covid_found = lsoup.find('div',id='COVID19')
        if covid_found:
            covid.append(1)
        else:
            covid.append(0)
        
        try:
            descparent = lsoup.find('div', attrs={"class":"eZQpm _T Z BB"})
            desctag = descparent.find('div', attrs={"class":"pIRBV _T"})
            description.append(desctag.getText())
        except Exception as e:
#             print(e)
            description.append(None)
    
        try:
            amtag = lsoup.find('div', attrs={"class":"ccdzg S5 b Pf ME"})
            if amtag.getText()=='Property amenities':
                amenitiestag = amtag.find_next_sibling('div', attrs={"class":"exmBD K"})
                amenity = []
                for a in amenitiestag.find_all('div', attrs={"class":"bUmsU f ME H3 _c"}):
                    amenity.append(a.getText())
                amenities.append(amenity)
            else:
                amenities.append(None)
    
        except:
            amenities.append(None)
        
        try:
            rfeaturetag = amtag.find_next_sibling('div', attrs={"class":"ccdzg S5 b Pf ME"})
            if rfeaturetag.getText()=='Room features':
                featurestag = rfeaturetag.find_next_sibling('div', attrs={"class":"exmBD K"})
                rfeature=[]
            
                for a in featurestag.find_all('div', attrs={"class":"bUmsU f ME H3 _c"}):
                    rfeature.append(a.getText())
                rfeatures.append(rfeature)
            else:
                rfeatures.append(None)
        except:
            rfeatures.append(None)
        
        try:
            rtypetag = rfeaturetag.find_next_sibling('div', attrs={"class":"ccdzg S5 b Pf ME"})
            if rtypetag.getText()=='Room types':
                roomtypetag = rtypetag.find_next_sibling('div', attrs={"class":"exmBD K"})
                rtype=[]
            
                for a in roomtypetag.find_all('div', attrs={"class":"bUmsU f ME H3 _c"}):
                    rtype.append(a.getText())
                rtypes.append(rtype)
            else:
                rtypes.append(None)
        except:
            rtypes.append(None)
        
        try:
            classtag = lsoup.find('div', attrs={"class":"dyJVb Ci S2 H2 b Wf"})
            if classtag.getText()=='HOTEL CLASS':
                classtag = classtag.find_next_sibling('div', attrs={"class":"drcGn _R MC S4 _a H"})
                hclass.append(classtag.find('svg', attrs={"class":"TkRkB d H0"})['title'].split()[0])
            else:
                hclass.append(None)
        except:
            hclass.append(None)
        
        try:
            styletag = classtag.find_next_sibling('div', attrs={"class":"dyJVb Ci S2 H2 b Wf"})
            if styletag.getText()=='HOTEL STYLE':
                styletag = styletag.find_next_siblings('div', attrs={"class":"drcGn _R MC S4 _a H"})
                style = []
                for b in styletag:
                    style.append(b.getText())
                hstyle.append(style)
            else:
                hstyle.append(None)
        except:
            hstyle.append(None)
        
        
        try:
            websitetag = lsoup.find('span', attrs={"class":"bhLNb ceIOZ GlpQN"})
            if websitetag.getText()=='Visit hotel website':
                website.append(1)
            else:
                website.append(0)
            
        except:
            website.append(0)
        
        try:
            offertag = lsoup.find('span', attrs={"class":"blOfferLead eyYtu b"})
            if offertag.getText()=='Special Offer:' or offertag.getText()=='Special Rate:':
                offers.append(1)
            else:
                offers.append(0)
            
        except:
            offers.append(0)
    
    
        ratetags = lsoup.find_all('div', attrs={"class":"cmZRz f"})
        try:
            locationrating.append(int(ratetags[0].find('span')['class'][1].replace('bubble_', ''))/10)
        except:
            locationrating.append(None)
    
        try:
            cleanrating.append(int(ratetags[1].find('span')['class'][1].replace('bubble_', ''))/10)
        except:
            cleanrating.append(None)
        
        try:
            servicerating.append(int(ratetags[2].find('span')['class'][1].replace('bubble_', ''))/10)
        except:
            servicerating.append(None)
        
        try:
            valuetag = lsoup.find('div', attrs={"class":"cmZRz f dfnfs"})
            valuerating.append(int(valuetag.find('span')['class'][1].replace('bubble_', ''))/10)
        except:
            valuerating.append(None)
        
        try:
            walktag = lsoup.find('span', attrs={"class":"bpwqy dfNPK"})
            walkscore.append(int(walktag.getText()))
        except:
            walkscore.append(None)
        
        try:
            restotag = lsoup.find('span', attrs={"class":"bpwqy VyMdE"})
            restoscore.append(int(restotag.getText()))
        except:
            restoscore.append(None)
                         
        try:
            attracttag = lsoup.find('span', attrs={"class":"bpwqy eKwbS"})
            attractscore.append(int(attracttag.getText()))
        except:
            attractscore.append(None)
        
        image_cnt=0
        try:
            image_cnt=lsoup.find('span',class_='is-hidden-tablet fBySN').getText()
#             timage_cnt= lsoup.find('span',class_='cXdRV').getText().split('(')[1].split(')')[0]
#             img_count = int(image_cnt) - int(timage_cnt)
        except:
            image_cnt=0
        image_count.append(image_cnt)
    
            
    time.sleep(2)
    
    # Using Selenium to navigate to the next page
    
    try:
        driver.find_element_by_xpath('//*[@id="taplc_main_pagination_bar_hotels_less_links_v2_0"]/div/div/div/span[2]').click()
    except Exception as e:
        print(e)
        break
    
    
    if count==1:
        break
    
    count+=1

driver.quit()
    
#     count+=1
#     print(count)

## Converting to a dataframe

In [ ]:
df = pd.DataFrame(list(zip(*[title, rank, rating, review, amenities, rfeatures, rtypes, hclass, hstyle, description, website, offers, covid, locationrating, cleanrating, valuerating, servicerating, walkscore, restoscore, attractscore, image_count, price])), 
                  columns = ['Title', 'Rank', 'Rating', 'Reviews', 'Amenities', 'RoomFeatures', 'RoomTypes', 'Class', 'Style', 'Description', 'Website', 'Offers', 'Covid', 'Location', 'Cleanliness', 'Value', 'Service', 'Walkscore', 'Restaurants', 'Attractions', 'ImageCount', 'Price'], dtype = str)

In [ ]:
df

In [ ]:
df.to_csv('ScrapedHotels2.csv', index=False)